## one prompt
```
第一个任务:
编写一个函数名为 get_breachlist，该函数需要导入 requests、PyQuery、pandas等库。该函数的输入参数为url，实现以下内容：
a) 使用 requests、PyQuery、pandas等库访问给定的 url，解析其中的 table标签，并将其转换为 Pandas DataFrame，将其保存为名为 df 的变量。如果 DataFrame 变量 df 是多级索引的，则删除第0层的 columns 索引。
b) 创建 href_dic 字典，首先获取 table 标签下的所有子级 a 标签的文本值作为键，获取 a 标签的 href 属性值并加上前缀 https://resources.hse.gov.uk/convictions-history/breach/作为值。
c) 将 href_dic 字典转换为 DataFrame，变量名为 href_df，字典的key值作为第一列, value 作为第二列, href_df 的第一列名要求与变量 df 的第一列一致.
d) 将变量 df 与变量 href_df 按第一列对齐，进行交集合并，作为 get_breachlist 的返回值。
e) 对函数进行修改，让其更加稳健。

第二个任务:
url = 'https://resources.hse.gov.uk/convictions-history/breach/breach_list.asp?PN=1&ST=B&EO=LIKE&SN=F&SF=DN&SV=&SO=DHD'
url = 'https://resources.hse.gov.uk/convictions-history/breach/breach_list.asp?PN=2&ST=B&EO=LIKE&SN=F&SF=DN&SV=&SO=DHD'
url = 'https://resources.hse.gov.uk/convictions-history/breach/breach_list.asp?PN=3&ST=B&EO=LIKE&SN=F&SF=DN&SV=&SO=DHD'
根据上面已给出的 3个url ,分析规律并创建一个 generate_url 函数,实现生成对应的 url 字符串, 利用该方法生成 410 个网址并且 保存在 urlist 数组中.

第三个任务:
使用多线程方法,线程数量为100, 执行 get_breachlist 方法, 将数组 urlist 所有元素作为参数传入,注意传入 urlist 与保存返回值时都要用到线程锁,返回值每增加100个时, 输入提示

当以上任务完成后,将所有获取到的返回值整理成一个dataframe,重置索引去重,排序操作后以 pickel 的格式保存至本地
```

In [ ]:
# 第一个任务
import requests
from pyquery import PyQuery as pq
import pandas as pd

def get_breachlist(url):
    # 使用 requests 库访问给定的 url
    response = requests.get(url)
    # 使用 PyQuery 库解析其中的 table 标签
    doc = pq(response.text)
    table = doc('table')
    # 将 table 标签转换为 Pandas DataFrame，将其保存为名为 df 的变量
    df = pd.read_html(str(table))[0]
    # 如果 DataFrame 变量 df 是多级索引的，则删除第0层的 columns 索引
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)
    # 创建 href_dic 字典，首先获取 table 标签下的所有子级 a 标签的文本值作为键，获取 a 标签的 href 属性值并加上前缀 https://resources.hse.gov.uk/convictions-history/breach/作为值
    href_dic = {}
    for a in table.find('a'):
        href_dic[pq(a).text()] = 'https://resources.hse.gov.uk/convictions-history/breach/' + pq(a).attr('href')
    # 将 href_dic 字典转换为 DataFrame，变量名为 href_df，字典的key值作为第一列, value 作为第二列, href_df 的第一列名要求与变量 df 的第一列一致
    href_df = pd.DataFrame(list(href_dic.items()), columns=[df.columns[0], 'href'])
    # 将变量 df 与变量 href_df 按第一列对齐，进行交集合并，作为 get_breachlist 的返回值
    result = pd.merge(df, href_df, on=df.columns[0], how='inner')
    return result

# 对函数进行修改，让其更加稳健
def get_breachlist(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(e)
        return None
    doc = pq(response.text)
    table = doc('table')
    try:
        df = pd.read_html(str(table))[0]
    except ValueError:
        print('No tables found')
        return None
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)
    href_dic = {}
    for a in table.find('a'):
        href_dic[pq(a).text()] = 'https://resources.hse.gov.uk/convictions-history/breach/' + pq(a).attr('href')
    href_df = pd.DataFrame(list(href_dic.items()), columns=[df.columns[0], 'href'])
    result = pd.merge(df, href_df, on=df.columns[0], how='inner')
    return result

# 第二个任务
def generate_url(page_num):
    return f'https://resources.hse.gov.uk/convictions-history/breach/breach_list.asp?PN={page_num}&ST=B&EO=LIKE&SN=F&SF=DN&SV=&SO=DHD'

urlist = [generate_url(i) for i in range(1, 411)]

# 第三个任务
import threading

lock = threading.Lock()
results = []

def get_breachlist_thread(url):
    result = get_breachlist(url)
    with lock:
        results.append(result)
        if len(results) % 100 == 0:
            print(f'{len(results)} urls processed')

threads = []
for url in urlist:
    t = threading.Thread(target=get_breachlist_thread, args=(url,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

# 将所有获取到的返回值整理成一个dataframe,重置索引去重,排序操作后以 pickel 的格式保存至本地
df = pd.concat(results, ignore_index=True)
df.drop_duplicates(inplace=True)
df.sort_values(by=df.columns[0], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_pickle('breachlist.pkl')

task 1
用pandas 读取本地目录下 breachlist.pkl 文件 ,保存为变量df
取出 df 的最后一列,转为list类型 保存为 urlist

task 2
创建一个 exp_excel 方法传入参数是 url,实现以下功能:
1 先创建局部变量 dflist cdlist  2个空数组 
2 再创建一个子方法 get_datails ,传入参数是 url: 实现以下内容
a) 声明使用 局部变量 dflist  与 cdlist  
b) 先用request , pyquery 获取 url 内容并解析 table 标签, 并将其转换为 DataFrame, 类型数据 并存入局部变量 dflist中

c) 另外, 我需要你遍历 table 标签下的所有子级 a 标签, 判断该标签的href  属性值是否包含 "case_details" 或 "defendant_details",如果是则将属性值的 ".." 替换为 "https://resources.hse.gov.uk/convictions-history", 
d) 当a标签 href 属性值包含 "case_details" 或  "defendant_details" 的情况时判断是否已经在 cdlist  数组中, 如果已存在则跳过,如果不存在则先存入 cdlist  数组,再执行 get_datails 方法,;
e) 如果不包含 "case_details" 或 "defendant_details" 字符的情况下, 跳过 ; 
到此, get_datails 定义结束

exp_excel  方法在上述的任何情况只要解析失败或异常时,请抛出异常返回0,  一切正常情况下 exp_excel  方法 的返回值是  dflist 元组

In [ ]:
# Task 1
import pandas as pd

# Read the breachlist.pkl file and save it as a dataframe variable df
df = pd.read_pickle('breachlist.pkl')

# Get the last column of df and convert it to a list, save it as urlist
urlist = df.iloc[:, -1].tolist()

# Task 2
import requests
from pyquery import PyQuery as pq

def exp_excel(url):
    # Create two empty arrays dflist and cdlist
    dflist = []
    cdlist = []
    
    # Define the get_datails function
    def get_datails(url):
        # Use the dflist and cdlist variables defined in the parent function
        nonlocal dflist, cdlist
        
        # Use requests and pyquery to get the content of the url and parse the table tag, then convert it to a DataFrame and save it to the dflist variable
        try:
            html = requests.get(url).text
            doc = pq(html)
            table = doc('table')
            df = pd.read_html(str(table))[0]
            dflist.append(df)
        except:
            raise Exception('Failed to parse table')
        
        # Traverse all child a tags under the table tag, and determine whether the href attribute value of the tag contains "case_details" or "defendant_details". If it does, replace ".." in the attribute value with "https://resources.hse.gov.uk/convictions-history"
        for a in table.find('a'):
            href = a.attrib.get('href')
            if href and ('case_details' in href or 'defendant_details' in href):
                href = href.replace('..', 'https://resources.hse.gov.uk/convictions-history')
                
                # If the href attribute value of the a tag contains "case_details" or "defendant_details", check whether it is already in the cdlist array. If it exists, skip it. If it does not exist, first save it to the cdlist array, and then execute the get_datails method.
                if href not in cdlist:
                    cdlist.append(href)
                    get_datails(href)
            else:
                # If the href attribute value of the a tag does not contain "case_details" or "defendant_details", skip it.
                continue
    
    # Call the get_datails function with the input url
    try:
        get_datails(url)
    except:
        raise Exception('Failed to get details')
    
    # Return the dflist tuple if everything is normal, or return 0 if an exception is thrown during execution
    return tuple(dflist) if dflist else 0


代码内容新增加, 写一个多线程任务, 对 urlist 中所有元素执行 exp_excel 方法, 
记录所有urlist 数组中元素执行的是否成功, 如果不成功能的重新传入执行,线程数量设定为50 , 
方法返回结果要用线程锁, 保存至reslist 数组中, 线程成功执行每100条 打印一次通知

In [ ]:

import threading

# Define a lock variable
lock = threading.Lock()

# Define a function to execute the exp_excel method and save the result to the reslist array
def execute_exp_excel(url):
    # Use the global variables urlist and reslist
    global urlist, reslist
    
    # Call the exp_excel method with the input url
    result = exp_excel(url)
    
    # Use the lock variable to ensure thread safety when modifying the reslist array
    with lock:
        # If the result is not 0, append it to the reslist array
        if result != 0:
            reslist.append(result)
        # If the result is 0, try to execute the exp_excel method again, up to 3 times
        else:
            for i in range(3):
                result = exp_excel(url)
                if result != 0:
                    reslist.append(result)
                    break
    
    # Print a notification every 100 successful executions
    if len(reslist) % 100 == 0:
        print('Successfully executed {} urls'.format(len(reslist)))

# Define a function to create and start threads
def create_and_start_threads():
    # Use the global variables urlist and reslist
    global urlist, reslist
    
    # Create a list to store threads
    threads = []
    
    # Create and start threads for each url in urlist
    for url in urlist:
        # Create a thread and add it to the threads list
        t = threading.Thread(target=execute_exp_excel, args=(url,))
        threads.append(t)
        
        # If the number of threads in the threads list is equal to 50, start all threads and clear the threads list
        if len(threads) == 50:
            for thread in threads:
                thread.start()
            for thread in threads:
                thread.join()
            threads.clear()
    
    # If there are any remaining threads in the threads list, start and join them
    if threads:
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()

# Create an empty reslist array
reslist = []

# Call the create_and_start_threads function to execute the exp_excel method for each url in urlist using multiple threads
create_and_start_threads()


用pickle 的方法 保存 reslist 变量到本地


In [ ]:
import pickle

# 用pickle.dump()方法将relist变量保存到本地
with open('reslist.pickle', 'wb') as f:
    pickle.dump(reslist, f)


读取

In [ ]:
import pickle
# Open the file in read mode
with open('reslist.pickle', 'rb') as f:
    # Load the data from the file
    ee = pickle.load(f)
len(ee)